In [1]:
import pandas as pd
import numpy as np

In [10]:
# Dataset original
# df = pd.read_csv('data/esets_lung_exprs_genes.txt',sep="\t",header=None,index_col=0)
# Dataset original-balanceado
df = pd.read_csv('Data\dataset_smoted.csv',index_col=0)

In [12]:
# Separación de data de entrenamiento - Data original balanceada
from sklearn.model_selection import train_test_split

X = df.drop('Tumor', axis=1)
y = df['Tumor']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=2)

In [13]:
# RadomForest
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
rfcla = RandomForestClassifier()
rfcla

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [15]:
# Un hiperparámetro importante es **n_estimators**, 
# que es la cantidad de árboles que se deben entrenar. 
# Debe ser un valor lo suficientemente grande como para que el error parezca haberse "estabilizado".
# Listar los valores a probar
from sklearn.model_selection import cross_val_score

estimator_range = range(10, 310, 10)

# Definir un lista para almacenar los promedios de los Auc por cada valor de "max_depth"
AUC_scores = []

# Usa una validación cruzada de 5 folds con cada valor del "n_estimators"
for estimator in estimator_range:
    rfcla = RandomForestClassifier(n_estimators=estimator, random_state=1, n_jobs=-1)
    AUC_score = cross_val_score(rfcla, X_train, y_train.values.ravel(), cv=5, scoring='roc_auc')
    AUC_scores.append(np.mean(AUC_score))
    #print(estimator)

In [16]:
# Graficar n_estimators (eje x) versus AUC (eje y)
import matplotlib.pyplot as plt 

plt.plot(estimator_range, AUC_scores)
plt.xlabel('n_estimators')
plt.ylabel('AUC')

Text(0, 0.5, 'AUC')

In [17]:
sorted(zip(AUC_scores, estimator_range), reverse=True)[0]

(0.99922913293449, 210)

In [20]:
# max_features=40  
# and n_estimators=210 mejor valor calculado
rfreg = RandomForestRegressor(n_estimators=210, max_features=40, oob_score=True, random_state=1)
rfreg.fit(X, y)
    


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=40, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=210, n_jobs=None, oob_score=True, random_state=1,
           verbose=0, warm_start=False)

In [25]:
# KappaScore

from sklearn.metrics import cohen_kappa_score

y_predict = rfreg.predict(X_test)

tup_y_predict = list(y_predict)
tup_y_train = list(np.transpose(y_test))

tup_y_predict = [int(i) for i in tup_y_predict]
cohen_kappa_score(y_test,tup_y_predict)

0.18150359371706104